In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras import backend as K

Using TensorFlow backend.


In [2]:
# allocates GPU memory based on runtime allocations. doesn't releases memory because of potential memory fragmentation
# without this option, getting images will not work
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [21]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 1

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [5]:
# this is the augmentation configuration for training
train_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [6]:
# this is the augmentation configuration we will use for testing:
test_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size)

Found 358 images belonging to 2 classes.


In [22]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 20 images belonging to 2 classes.


In [11]:
model.fit_generator(
    train_generator,
#     steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Epoch 1/50
125/125 [==============================] - 18s 142ms/step - loss: 0.6392 - acc: 0.7077 - val_loss: 0.7085 - val_acc: 0.4500
Epoch 2/50
125/125 [==============================] - 16s 125ms/step - loss: 0.5420 - acc: 0.7337 - val_loss: 0.7096 - val_acc: 0.5000
Epoch 3/50
125/125 [==============================] - 16s 124ms/step - loss: 0.4570 - acc: 0.7866 - val_loss: 0.6795 - val_acc: 0.6500
Epoch 4/50
125/125 [==============================] - 16s 125ms/step - loss: 0.4047 - acc: 0.8190 - val_loss: 0.7042 - val_acc: 0.6000
Epoch 5/50
125/125 [==============================] - 16s 125ms/step - loss: 0.3668 - acc: 0.8342 - val_loss: 0.8932 - val_acc: 0.6000
Epoch 6/50
125/125 [==============================] - 16s 126ms/step - loss: 0.3128 - acc: 0.8616 - val_loss: 0.8483 - val_acc: 0.7000
Epoch 7/50
125/125 [==============================] - 16s 125ms/step - loss: 0.2836 - acc: 0.8768 - val_loss: 0.8060 - val_acc: 0.6000
Epoch 8/50
125/125 [==============================] - 1

In [23]:
loaded_model = Sequential()

loaded_model.add(Conv2D(32, (3, 3), input_shape=input_shape))
loaded_model.add(Activation('relu'))
loaded_model.add(MaxPooling2D(pool_size=(2, 2)))

loaded_model.add(Conv2D(32, (3, 3)))
loaded_model.add(Activation('relu'))
loaded_model.add(MaxPooling2D(pool_size=(2, 2)))

loaded_model.add(Conv2D(64, (3, 3)))
loaded_model.add(Activation('relu'))
loaded_model.add(MaxPooling2D(pool_size=(2, 2)))

loaded_model.add(Flatten())
loaded_model.add(Dense(64))
loaded_model.add(Activation('relu'))
loaded_model.add(Dropout(0.5))
loaded_model.add(Dense(1))
loaded_model.add(Activation('sigmoid'))

loaded_model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

loaded_model.load_weights('first_try.h5')

In [32]:
prediction = loaded_model.predict_generator(validation_generator, verbose=1)

20/20 [==============================] - 0s 9ms/step


In [25]:
prediction

array([[8.8824493e-01],
       [1.0000000e+00],
       [9.0835071e-01],
       [3.6239746e-01],
       [1.0088424e-02],
       [1.9106631e-01],
       [7.4745190e-01],
       [9.7184932e-01],
       [1.0000000e+00],
       [8.5396147e-01],
       [3.5945636e-01],
       [6.1313415e-01],
       [1.0000000e+00],
       [1.0000000e+00],
       [5.6815398e-01],
       [9.9895072e-01],
       [8.7238318e-01],
       [1.0182740e-01],
       [9.9976343e-01],
       [3.3065625e-08]], dtype=float32)

In [26]:
print(y_true)
print(y_pred)

[0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1]
[[False]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [False]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [False]
 [ True]
 [ True]
 [ True]]


In [27]:
from sklearn.metrics import confusion_matrix
import numpy as np

y_true = np.array([0] * 9 + [1] * 11)
y_pred = prediction > 0.5

confusion_matrix(y_true, y_pred)

array([[3, 6],
       [3, 8]], dtype=int64)

In [26]:
prediction.shape

(20, 1)

In [28]:
y_pred

array([[ True],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [False]])